In [2]:
import subprocess
import time
import socket
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def check_port_open(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('localhost', port))
    return result == 0  # Port is open if result is 0

# Khởi động Chrome với remote-debugging port
subprocess.Popen([
    r'C:\Program Files\Google\Chrome\Application\chrome.exe',
    '--remote-debugging-port=8797',
    r'--user-data-dir=D:\Crawling and Scraping\chromedriver\win64\134.0.6998.165\chromedriver-win32\localhost'  # Profile riêng
])

# Đợi cho đến khi port 8797 mở
while not check_port_open(8797):
    time.sleep(0.5)

# Kết nối bằng Selenium
option = Options()
option.add_experimental_option("debuggerAddress", "localhost:8797")

driver = webdriver.Chrome(options=option)
driver.get("https://www.udemy.com/course/python-django-the-complete-django-web-development-course/learn/lecture/33483924#questions")


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [81]:
button_load_more_xpath = '//*[@id="tabs--3-content-3"]/div/div/div[3]/div[2]/div[2]/button'
button = driver.find_element("xpath", button_load_more_xpath)
button.click()

In [75]:
def clickButton():
    button_load_more_xpath = '//*[@id="tabs--3-content-3"]/div/div/div[3]/div[2]/div[2]/button'
    try:
        button = driver.find_element("xpath", button_load_more_xpath)
        print("Chuẩn bị click")
        button.click()
    except Exception as e:
        print("Đã load hết bình luận")

In [76]:
clickButton()
time.sleep(5)

clickButton()
time.sleep(5)

clickButton()


Đã load hết bình luận
Đã load hết bình luận
Đã load hết bình luận


In [77]:
# Tìm tất cả các thẻ <a> có href chứa '/user'
user_links = driver.find_elements("xpath", '//*[@id="tabs--3-content-3"]/div/div/div[3]/div[2]/div[1]//a[contains(@href, "/user")]')

# Duyệt và in ra giá trị href
for link in user_links:
    href = link.get_attribute("href")
    print(href)

In [78]:
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

# Hàm click nút "Hiển thị thêm"
def clickButton(driver: WebDriver) -> bool:
    button_load_more_xpath = '//*[@id="tabs--3-content-3"]/div/div/div[3]/div[2]/div[2]/button'
    try:
        button = driver.find_element("xpath", button_load_more_xpath)
        print("[CLICK] Nhấn 'Hiển thị thêm' để load thêm người dùng...")
        button.click()
        time.sleep(2)  # Đợi nội dung mới load ra
        return True
    except Exception as e:
        print("[INFO] Không còn nút 'Hiển thị thêm' hoặc lỗi khi click:", e)
        return False

# Hàm lấy danh sách user URL hiện tại
def get_url_user(driver: WebDriver) -> list:
    user_url_list = []
    try:
        user_links = driver.find_elements(
            "xpath", 
            '//*[@id="tabs--3-content-3"]/div/div/div[3]/div[2]/div[1]//a[contains(@href, "/user")]'
        ) 
        for link in user_links:
            href = link.get_attribute("href")
            if href:
                user_url_list.append(href)
    except NoSuchElementException as e:
        print("[ERROR] Không tìm thấy phần tử:", e)
    except Exception as e:
        print("[ERROR] Đã xảy ra lỗi:", e)
    return user_url_list

# Hàm tổng hợp crawl toàn bộ user URLs
def crawl_all_user_urls(driver: WebDriver) -> list:
    all_user_urls = set()
    round_count = 1

    while True:
        print(f"\n[ROUND {round_count}] Đang lấy danh sách user URLs...")
        user_urls = get_url_user(driver)
        before_count = len(all_user_urls)
        all_user_urls.update(user_urls)
        after_count = len(all_user_urls)

        new_count = after_count - before_count
        print(f"[INFO] Tìm được {new_count} URL mới. Tổng: {after_count} URL.")

        if not clickButton(driver):
            print("[DONE] Đã load hết tất cả user URLs 👌")
            break

        round_count += 1
        time.sleep(1)  # Có thể điều chỉnh thời gian tùy tốc độ website

    return list(all_user_urls)

In [84]:
all_user_links = crawl_all_user_urls(driver)

# In kết quả:
print(f"\n✅ Tổng số user URLs thu thập được: {len(all_user_links)}")
for url in all_user_links:
    print(url)


[ROUND 1] Đang lấy danh sách user URLs...
[INFO] Tìm được 5 URL mới. Tổng: 5 URL.
[CLICK] Nhấn 'Hiển thị thêm' để load thêm người dùng...

[ROUND 2] Đang lấy danh sách user URLs...
[INFO] Tìm được 5 URL mới. Tổng: 10 URL.
[CLICK] Nhấn 'Hiển thị thêm' để load thêm người dùng...

[ROUND 3] Đang lấy danh sách user URLs...
[INFO] Tìm được 4 URL mới. Tổng: 14 URL.
[CLICK] Nhấn 'Hiển thị thêm' để load thêm người dùng...

[ROUND 4] Đang lấy danh sách user URLs...
[INFO] Tìm được 5 URL mới. Tổng: 19 URL.
[CLICK] Nhấn 'Hiển thị thêm' để load thêm người dùng...

[ROUND 5] Đang lấy danh sách user URLs...
[INFO] Tìm được 5 URL mới. Tổng: 24 URL.
[CLICK] Nhấn 'Hiển thị thêm' để load thêm người dùng...

[ROUND 6] Đang lấy danh sách user URLs...
[INFO] Tìm được 5 URL mới. Tổng: 29 URL.
[CLICK] Nhấn 'Hiển thị thêm' để load thêm người dùng...

[ROUND 7] Đang lấy danh sách user URLs...
[INFO] Tìm được 4 URL mới. Tổng: 33 URL.
[CLICK] Nhấn 'Hiển thị thêm' để load thêm người dùng...

[ROUND 8] Đang lấy d

In [85]:
all_user_links

['https://www.udemy.com/user/joydip-roychowdhury-7/',
 'https://www.udemy.com/user/sanket-kachare-sk/',
 'https://www.udemy.com/user/desmond-793/',
 'https://www.udemy.com/user/guido-luis-dalla-vecchia/',
 'https://www.udemy.com/user/shivam-ghorpade-5/',
 'https://www.udemy.com/user/abdul-hannan-121/',
 'https://www.udemy.com/user/fred-k-parsons/',
 'https://www.udemy.com/user/fadyghalayiny3/',
 'https://www.udemy.com/user/mohab-mohammed-11/',
 'https://www.udemy.com/user/nazrul-islam-anik/',
 'https://www.udemy.com/user/kamalakar-106/',
 'https://www.udemy.com/user/hendrik-botha/',
 'https://www.udemy.com/user/sanjay-prajapati-111/',
 'https://www.udemy.com/user/miguel-banegas2/',
 'https://www.udemy.com/user/caelan-quigley-2/',
 'https://www.udemy.com/user/axmed-badal/',
 'https://www.udemy.com/user/tatoh-modest-2/',
 'https://www.udemy.com/user/ibraheem-omikunle/',
 'https://www.udemy.com/user/suleiman-abdinasir-jama/',
 'https://www.udemy.com/user/rounak-runwal-3/',
 'https://www.u

In [87]:
import pandas as pd

data = {
'python-django-the-complete-django-web-development-course': ['all_user_links'],
'user_url': [all_user_links] # Giữ nguyên danh sách trong một ô
}

df = pd.DataFrame(data)

df.to_csv('user_links.csv', index=False)